In [1]:
import pandas as pd 
from pathlib import Path
from src.panspace.utils import LogInfo
loginfo = LogInfo()    

## USRBINTIME
___

RAM Usage, User Time

In [2]:
folder_logs = "/data/bacteria/experiments/autoencoders/6mer/01_22_2024-1/cross-validation/logs"
list_logs = list(Path(folder_logs).rglob("*log"))

In [3]:
list_logs[0]

PosixPath('/data/bacteria/experiments/autoencoders/6mer/01_22_2024-1/cross-validation/logs/train_mean_squared_error-relu-relu-5-fold.log')

In [4]:
info = []
for log in list_logs:
    info_ = loginfo(log)
    # info_["log"] = log

    # info from path log
    datapath = Path(log).stem.split("_") 
    rule = "_".join(datapath[:2])
    fold = datapath[-2]


    info_["rule"] = rule
    if "fold" in datapath[-1]:
        _ , hidden_activation, output_activation , kfold , _ = datapath[-1].split("-")

        info_["fold"] =  kfold
        info_["hidden_activation"] = hidden_activation
        info_["output_activation"] = output_activation
        info_["rule"] = rule
        
    info.append(info_)

In [5]:
_, hidden_activation, output_activation , kfold , _ = datapath[-1].split("-")

In [6]:
df_usrbintime = pd.DataFrame(info)
df_usrbintime["Max RAM (GB)"] = df_usrbintime["Maximum resident set size (kbytes)"] / 1_000_000

In [7]:
import numpy as np
df_usrbintime.groupby(["rule"])["User time (seconds)"].describe()

,count,mean,std,min,25%,50%,75%,max
rule,,,,,,,,
create_index,5.0,162.340,1.645190,159.71,161.88,162.81,163.50,163.80
extract_encoder,5.0,1.470,0.047434,1.41,1.43,1.49,1.50,1.52
test_index,5.0,310.224,5.010437,302.70,307.47,313.21,313.27,314.47
train_mean,5.0,4972.458,689.926704,4594.95,4654.70,4688.57,4720.23,6203.84


In [8]:
df_usrbintime.groupby(["rule"])["Max RAM (GB)"].describe()

,count,mean,std,min,25%,50%,75%,max
rule,,,,,,,,
create_index,5.0,1.328654,0.056474,1.228800,1.343008,1.349952,1.355032,1.366480
extract_encoder,5.0,0.571396,0.001977,0.568608,0.571088,0.571328,0.571804,0.574152
test_index,5.0,2.018552,0.015372,1.994372,2.015764,2.020396,2.027004,2.035224
train_mean,5.0,0.990182,0.018710,0.970660,0.974428,0.988076,1.003044,1.014700


## Accuracy test 


In [9]:
import json
import pandas as pd

from pathlib import Path
from collections import Counter, namedtuple, defaultdict

from sklearn.metrics import (
    precision_score, 
    recall_score,
    accuracy_score, 
    balanced_accuracy_score
    )

### add consensus label to predictions

In [360]:
kfold = 1
loss="binary_crossentropy"
hidden_activation="relu"
output_activation="sigmoid"
path_exp= Path(f"/data/bacteria/experiments/autoencoders/6mer/01_22_2024-2/cross-validation/{loss}-{hidden_activation}-{output_activation}-{kfold}-fold")

In [361]:
path_train = path_exp.parent.joinpath(f"train_{kfold}-fold.txt")
path_test = path_exp.parent.joinpath(f"test_{kfold}-fold.txt")

In [362]:
# collect info in a dataframe
InfoLabels = namedtuple("InfoLabels",["label","dataset","count"])
counts = dict()
data = []
for path in [path_train, path_test]:#["train","val","test"]:
    ds = path.stem.split("_")[0]
    with open(path,"r") as fp:
        labels = []
        for line in fp.readlines():
            label = line.replace("\n","").split("\t")[-1]
            labels.append(label) 

    # add count of labels 
    counts = Counter(labels)
    for sp, c in counts.items():
        data.append(
            InfoLabels(sp, ds, c)
        )

df_infolabels = pd.DataFrame(data)

# get dict with count for the test set for later evaluation
counts_test = dict()
for idx, sp, ds, count in df_infolabels.query("dataset == 'test'").to_records("record"):
    counts_test[sp] = count

counts_train = defaultdict(int)
for idx, sp, ds, count in df_infolabels.query("dataset != 'test'").to_records("record"):
    counts_train[sp] += count

In [363]:
with open(path_exp.joinpath("faiss-embeddings/labels.json") ,"r") as fp:
    labels_by_sampleid=json.load(fp)
labels_index = set(list(labels_by_sampleid.values()))

In [364]:
df = pd.read_csv(path_exp.joinpath("test/query_results.csv"))
df["label_in_index"] = df["ground_truth"].apply(lambda x: x in labels_index)

In [365]:
cols_labels = [c for c in df.columns if c.startswith("label") and not c.endswith("index")]
N = 10 # depending on this, remove from the analysis species with less than N representatives in the index
cols_N = [c for c in cols_labels if int(c.split("_")[-1])<N]

In [366]:
df["consensus"] = df[cols_N].apply(lambda row: Counter(row).most_common(1)[0][0],axis=1)

### how many ground truth are not in the labels indexed?

In [367]:
df["allow_query"] = df["ground_truth"].apply(lambda x: True if counts_train.get(x,0)>N else False)
df

,Unnamed: 0,ground_truth,sample_id_query,sample_id_0,label_0,distance_to_0,sample_id_1,label_1,distance_to_1,sample_id_2,...,distance_to_7,sample_id_8,label_8,distance_to_8,sample_id_9,label_9,distance_to_9,label_in_index,consensus,allow_query
0,0,neisseria_gonorrhoeae,SAMN04624666,SAMEA104081187,neisseria_gonorrhoeae,0.000067,SAMEA3672287,neisseria_gonorrhoeae,0.000086,SAMEA3431930,...,0.000101,SAMN04625250,neisseria_gonorrhoeae,0.000101,SAMEA3497593,neisseria_gonorrhoeae,0.000102,True,neisseria_gonorrhoeae,True
1,1,escherichia_coli,SAMN09784558,SAMN06029321,escherichia_coli,0.000084,SAMN08144280,escherichia_coli,0.000084,SAMEA4062301,...,0.000105,SAMN08963255,escherichia_coli,0.000105,SAMN03983338,escherichia_coli,0.000106,True,escherichia_coli,True
2,2,salmonella_enterica,SAMN09788966,SAMN04575359,salmonella_enterica,0.000044,SAMN09430533,salmonella_enterica,0.000046,SAMN06842097,...,0.000049,SAMN03112858,salmonella_enterica,0.000049,SAMN06285102,salmonella_enterica,0.000052,True,salmonella_enterica,True
3,3,salmonella_enterica,SAMN07159320,SAMN07138696,salmonella_enterica,0.000091,SAMN06890573,salmonella_enterica,0.000094,SAMN10240765,...,0.000113,SAMN09727060,salmonella_enterica,0.000113,SAMN02929626,salmonella_enterica,0.000115,True,salmonella_enterica,True
4,4,salmonella_enterica,SAMN07227128,SAMEA80635918,salmonella_enterica,0.000030,SAMEA2148364,salmonella_enterica,0.000035,SAMEA4759513,...,0.000048,SAMEA104162062,salmonella_enterica,0.000048,SAMN09294259,salmonella_enterica,0.000049,True,salmonella_enterica,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132274,132274,salmonella_enterica,SAMN07731205,SAMN10396305,salmonella_enterica,0.000030,SAMN09655213,salmonella_enterica,0.000034,SAMN09501203,...,0.000039,SAMN08823363,salmonella_enterica,0.000041,SAMN09848275,salmonella_enterica,0.000042,True,salmonella_enterica,True
132275,132275,neisseria_meningitidis,SAMEA2570241,SAMEA2570265,neisseria_meningitidis,0.000101,SAMEA3503529,neisseria_meningitidis,0.000116,SAMEA2074075,...,0.000144,SAMEA2074102,neisseria_meningitidis,0.000144,SAMEA3503222,neisseria_meningitidis,0.000145,True,neisseria_meningitidis,True
132276,132276,salmonella_enterica,SAMN03252939,SAMN08981889,salmonella_enterica,0.000040,SAMN07156430,salmonella_enterica,0.000040,SAMN04942592,...,0.000054,SAMN07312903,salmonella_enterica,0.000054,SAMN09396902,salmonella_enterica,0.000055,True,salmonella_enterica,True
132277,132277,mycobacterium_tuberculosis,SAMEA2535176,SAMN08629129,mycobacterium_tuberculosis,0.000033,SAMN09090641,mycobacterium_tuberculosis,0.000033,SAMEA3233432,...,0.000043,SAMEA2534475,mycobacterium_tuberculosis,0.000044,SAMN06055789,mycobacterium_tuberculosis,0.000045,True,mycobacterium_tuberculosis,True


In [368]:
df.query("allow_query==False").shape

(1518, 36)

In [369]:
df_metrics = df.query("allow_query == True").copy()
classes = df_metrics.ground_truth.unique()
classes = sorted(classes)
predicted = sorted(df_metrics.consensus.unique())

y_true, y_pred = df_metrics.ground_truth, df_metrics.consensus

accuracy_score(y_true, y_pred), balanced_accuracy_score(y_true, y_pred), len(classes), len(predicted)

/home/avila/micromamba/envs/train/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


(0.9854084933581113, 0.773964519817596, 543, 666)

In [332]:
df["is_correct"] = df.apply(lambda r: r["ground_truth"] == r["consensus"],axis=1)

In [333]:
df.query("is_correct==False")

,Unnamed: 0,ground_truth,sample_id_query,sample_id_0,label_0,distance_to_0,sample_id_1,label_1,distance_to_1,sample_id_2,...,sample_id_8,label_8,distance_to_8,sample_id_9,label_9,distance_to_9,label_in_index,consensus,allow_query,is_correct
32,32,erysipelothrix_rhusiopathiae,SAMD00093494,SAMN00828719,klebsiella_pneumoniae,0.035573,SAMN04527051,clostridium_butyricum,0.043954,SAMN00771407,...,SAMN03842090,negativicoccus_massiliensis,0.067281,SAMN07125784,dichelobacter_nodosus,0.067732,True,klebsiella_pneumoniae,True,False
86,86,porphyrobacter_ht-58-2,SAMN02745577,SAMN02744741,escherichia_coli,0.006007,SAMN02745236,escherichia_coli,0.006893,SAMN02745054,...,SAMN06314750,staphylococcus_aureus,0.024048,SAMN02745326,winogradskyella_sp._pc-19,0.026762,False,escherichia_coli,False,False
154,154,mycobacterium_sp._vkm_ac-1817d,SAMEA3448419,SAMEA2627011,mycolicibacterium_fortuitum,0.000110,SAMN04634203,mycolicibacterium_fortuitum,0.000129,SAMN04634272,...,SAMN07660343,mycolicibacterium_fortuitum,0.000194,SAMN07659996,mycolicibacterium_fortuitum,0.000199,True,mycolicibacterium_fortuitum,True,False
314,314,campylobacter_coli,SAMN02746106,SAMN05720327,sulfurimonas_gotlandica,0.051914,SAMN05720328,sulfurimonas_gotlandica,0.059357,SAMN05720361,...,SAMN02745436,sulfurimonas_gotlandica,0.083103,SAMN05720573,sulfurimonas_gotlandica,0.083481,True,sulfurimonas_gotlandica,True,False
328,328,enterobacter_cloacae,SAMEA4427127,SAMEA2273406,enterobacter_cloacae,0.000073,SAMN08932722,enterobacter_cloacae,0.000074,SAMEA2054045,...,SAMEA2273404,enterobacter_cloacae,0.000114,SAMEA2203674,enterobacter_hormaechei,0.000115,True,enterobacter_hormaechei,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132118,132118,prevotella_dentalis,SAMN02584977,SAMN00262626,prevotella_dentalis,0.001544,SAMN05761448,tannerella_sp._oral_taxon_hot-286,0.056348,SAMN05761449,...,SAMN02420388,tannerella_sp._oral_taxon_hot-286,0.078623,SAMEA29990668,faecalibacterium_prausnitzii,0.079153,True,tannerella_sp._oral_taxon_hot-286,False,False
132157,132157,flavisolibacter_tropicus,SAMN05444008,SAMN06163200,flavisolibacter_tropicus,0.000259,SAMN02199131,salmonella_enterica,0.059158,SAMN07621255,...,SAMN05421747,sphingobacterium_sp._21,0.082616,SAMN05216457,chitinophaga_pinensis,0.084817,True,chitinophaga_pinensis,False,False
132210,132210,nitrosomonas_ureae,SAMN05421510,SAMN08778374,nitrosomonas_ureae,0.000410,SAMN05216406,nitrosomonas_ureae,0.000708,SAMN08778363,...,SAMN08779447,nitrosomonas_sp._is79a3,0.035134,SAMN02786974,nitrosomonas_sp._is79a3,0.053639,True,nitrosomonas_sp._is79a3,False,False
132241,132241,streptomyces_cyaneogriseus,SAMD00012182,SAMN08125747,shewanella_piezotolerans,0.034624,SAMN08125754,shewanella_piezotolerans,0.035308,SAMN08125761,...,SAMN09399390,shewanella_bicestrii,0.052509,SAMN06299585,vibrio_shilonii,0.052531,True,shewanella_piezotolerans,False,False


In [334]:
df.query("is_correct==True")

,Unnamed: 0,ground_truth,sample_id_query,sample_id_0,label_0,distance_to_0,sample_id_1,label_1,distance_to_1,sample_id_2,...,sample_id_8,label_8,distance_to_8,sample_id_9,label_9,distance_to_9,label_in_index,consensus,allow_query,is_correct
0,0,escherichia_coli,SAMN04504389,SAMN07124441,escherichia_coli,0.000048,SAMN09934133,escherichia_coli,0.000072,SAMN06848979,...,SAMN05438631,escherichia_coli,0.000080,SAMN08638583,escherichia_coli,0.000080,True,escherichia_coli,True,True
1,1,acinetobacter_baumannii,SAMEA1569552,SAMEA1569419,acinetobacter_baumannii,0.000075,SAMN06007771,acinetobacter_baumannii,0.000093,SAMEA1466078,...,SAMN04993338,acinetobacter_baumannii,0.000108,SAMN04901697,acinetobacter_baumannii,0.000109,True,acinetobacter_baumannii,True,True
2,2,salmonella_enterica,SAMN08056719,SAMN08832519,salmonella_enterica,0.000021,SAMN02368002,salmonella_enterica,0.000021,SAMN03577009,...,SAMN02927340,salmonella_enterica,0.000026,SAMN09843238,salmonella_enterica,0.000026,True,salmonella_enterica,True,True
3,3,salmonella_enterica,SAMN08359571,SAMN07197453,salmonella_enterica,0.000014,SAMN06270090,salmonella_enterica,0.000022,SAMN08737177,...,SAMN07565860,salmonella_enterica,0.000025,SAMN06045208,salmonella_enterica,0.000026,True,salmonella_enterica,True,True
4,4,campylobacter_jejuni,SAMN09873581,SAMN04532351,campylobacter_jejuni,0.000031,SAMEA2275356,campylobacter_jejuni,0.000058,SAMEA2361287,...,SAMEA3391649,campylobacter_jejuni,0.000101,SAMEA2061903,campylobacter_jejuni,0.000101,True,campylobacter_jejuni,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132273,132273,escherichia_coli,SAMN09982189,SAMN01048125,escherichia_coli,0.000014,SAMN06650448,escherichia_coli,0.000021,SAMN04262805,...,SAMN04262688,escherichia_coli,0.000032,SAMN08865281,escherichia_coli,0.000034,True,escherichia_coli,True,True
132274,132274,streptococcus_pneumoniae,SAMEA807290,SAMN04504914,streptococcus_pneumoniae,0.000883,SAMEA2467767,streptococcus_pneumoniae,0.003111,SAMEA1023968,...,SAMEA2203901,streptococcus_pneumoniae,0.003465,SAMEA2467760,streptococcus_pneumoniae,0.003479,True,streptococcus_pneumoniae,True,True
132275,132275,salmonella_enterica,SAMN10170869,SAMN03946276,salmonella_enterica,0.000026,SAMN06891171,salmonella_enterica,0.000027,SAMN09489687,...,SAMN07551777,salmonella_enterica,0.000034,SAMN06244432,salmonella_enterica,0.000035,True,salmonella_enterica,True,True
132276,132276,clostridioides_difficile,SAMEA2239391,SAMN05208051,clostridioides_difficile,0.000029,SAMN04459192,clostridioides_difficile,0.000070,SAMN03473848,...,SAMEA1710751,clostridioides_difficile,0.000117,SAMN04458409,clostridioides_difficile,0.000130,True,clostridioides_difficile,True,True


In [370]:
df_metrics

,Unnamed: 0,ground_truth,sample_id_query,sample_id_0,label_0,distance_to_0,sample_id_1,label_1,distance_to_1,sample_id_2,...,distance_to_7,sample_id_8,label_8,distance_to_8,sample_id_9,label_9,distance_to_9,label_in_index,consensus,allow_query
0,0,neisseria_gonorrhoeae,SAMN04624666,SAMEA104081187,neisseria_gonorrhoeae,0.000067,SAMEA3672287,neisseria_gonorrhoeae,0.000086,SAMEA3431930,...,0.000101,SAMN04625250,neisseria_gonorrhoeae,0.000101,SAMEA3497593,neisseria_gonorrhoeae,0.000102,True,neisseria_gonorrhoeae,True
1,1,escherichia_coli,SAMN09784558,SAMN06029321,escherichia_coli,0.000084,SAMN08144280,escherichia_coli,0.000084,SAMEA4062301,...,0.000105,SAMN08963255,escherichia_coli,0.000105,SAMN03983338,escherichia_coli,0.000106,True,escherichia_coli,True
2,2,salmonella_enterica,SAMN09788966,SAMN04575359,salmonella_enterica,0.000044,SAMN09430533,salmonella_enterica,0.000046,SAMN06842097,...,0.000049,SAMN03112858,salmonella_enterica,0.000049,SAMN06285102,salmonella_enterica,0.000052,True,salmonella_enterica,True
3,3,salmonella_enterica,SAMN07159320,SAMN07138696,salmonella_enterica,0.000091,SAMN06890573,salmonella_enterica,0.000094,SAMN10240765,...,0.000113,SAMN09727060,salmonella_enterica,0.000113,SAMN02929626,salmonella_enterica,0.000115,True,salmonella_enterica,True
4,4,salmonella_enterica,SAMN07227128,SAMEA80635918,salmonella_enterica,0.000030,SAMEA2148364,salmonella_enterica,0.000035,SAMEA4759513,...,0.000048,SAMEA104162062,salmonella_enterica,0.000048,SAMN09294259,salmonella_enterica,0.000049,True,salmonella_enterica,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132274,132274,salmonella_enterica,SAMN07731205,SAMN10396305,salmonella_enterica,0.000030,SAMN09655213,salmonella_enterica,0.000034,SAMN09501203,...,0.000039,SAMN08823363,salmonella_enterica,0.000041,SAMN09848275,salmonella_enterica,0.000042,True,salmonella_enterica,True
132275,132275,neisseria_meningitidis,SAMEA2570241,SAMEA2570265,neisseria_meningitidis,0.000101,SAMEA3503529,neisseria_meningitidis,0.000116,SAMEA2074075,...,0.000144,SAMEA2074102,neisseria_meningitidis,0.000144,SAMEA3503222,neisseria_meningitidis,0.000145,True,neisseria_meningitidis,True
132276,132276,salmonella_enterica,SAMN03252939,SAMN08981889,salmonella_enterica,0.000040,SAMN07156430,salmonella_enterica,0.000040,SAMN04942592,...,0.000054,SAMN07312903,salmonella_enterica,0.000054,SAMN09396902,salmonella_enterica,0.000055,True,salmonella_enterica,True
132277,132277,mycobacterium_tuberculosis,SAMEA2535176,SAMN08629129,mycobacterium_tuberculosis,0.000033,SAMN09090641,mycobacterium_tuberculosis,0.000033,SAMEA3233432,...,0.000043,SAMEA2534475,mycobacterium_tuberculosis,0.000044,SAMN06055789,mycobacterium_tuberculosis,0.000045,True,mycobacterium_tuberculosis,True


In [335]:
precision = precision_score(y_true, y_pred, average=None, labels=classes)
recall = recall_score(y_true, y_pred, average=None, labels=classes)

/home/avila/micromamba/envs/train/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [372]:
data_metrics = []
Metrics = namedtuple("Metrics", ["label","n_queries", "n_index", "precision","recall"])
for sp, prec, rec in zip(classes, precision, recall):
    n_queries= counts_test[sp]
    n_index = counts_train[sp]
    data_metrics.append(
        Metrics(sp, n_queries, n_index, prec, rec)
    )

df_metrics = pd.DataFrame(data_metrics).sort_values(by=["recall"])
df_metrics.sort_values("label").sort_values(by="precision")

,label,n_queries,n_index,precision,recall
516,vibrio_fluvialis,4,15,0.0,0.000000
529,xanthomonas_fragariae,9,35,0.0,0.000000
501,tannerella_forsythia,2,18,0.0,0.000000
52,bacillus_thuringiensis,49,193,0.0,0.000000
53,bacillus_velezensis,9,23,0.0,0.000000
...,...,...,...,...,...
541,yoonia_vestfoldensis,3,11,1.0,0.988889
542,zunongwangia_profunda,4,27,1.0,0.666667
1,[eubacterium]_eligens,3,16,1.0,0.250000
517,vibrio_harveyi,6,16,1.0,1.000000


In [378]:
df_metrics.query(f"n_index>0")

,label,n_queries,n_index,precision,recall
501,tannerella_forsythia,2,18,0.0,0.0
516,vibrio_fluvialis,4,15,0.0,0.0
11,acinetobacter_defluvii,5,15,0.0,0.0
188,enterococcus_gilvus,13,36,0.0,0.0
67,bifidobacterium_longum,14,61,0.0,0.0
...,...,...,...,...,...
540,yersinia_ruckeri,8,50,1.0,1.0
522,vibrio_vulnificus,58,160,1.0,1.0
520,vibrio_shilonii,43,173,1.0,1.0
4,achromobacter_spanius,4,11,1.0,1.0


In [338]:
df_metrics.query(f"0<n_index<80 and precision > 0").sort_values(by=["n_queries","precision"])

,label,n_queries,n_index,precision,recall
123,candidatus_pelagibacter_ubique,1,25,0.142857,1.000000
190,erysipelotrichaceae_bacterium_i46,1,23,0.200000,1.000000
289,methylorubrum_extorquens,1,11,0.333333,1.000000
418,roseomonas_sp._fdaargos_362,1,14,0.333333,1.000000
408,rhizobium_etli,1,13,0.500000,1.000000
...,...,...,...,...,...
113,campylobacter_hyointestinalis,22,63,1.000000,0.954545
284,mesorhizobium_amorphae,23,70,0.720000,0.782609
482,streptococcus_parasanguinis,24,63,0.958333,0.958333
83,brucella_canis,27,50,1.000000,0.037037


In [339]:
df_metrics.query(f"precision > 0.9 and recall > 0.9")

,label,n_queries,n_index,precision,recall
137,citrobacter_freundii,64,284,0.906250,0.906250
15,acinetobacter_nosocomialis,33,84,1.000000,0.909091
357,paeniclostridium_sordellii,11,62,0.909091,0.909091
369,porphyromonas_gingivalis,22,78,0.952381,0.909091
396,pseudomonas_stutzeri,22,60,1.000000,0.909091
...,...,...,...,...,...
65,bifidobacterium_thermophilum,3,11,1.000000,1.000000
36,aliivibrio_fischeri,10,70,0.909091,1.000000
40,arcobacter_trophiarum,3,16,1.000000,1.000000
41,arthrobacter_sp._qxt-31,16,44,1.000000,1.000000
